In [ ]:
from nba_api.stats.endpoints import shotchartdetail, playercareerstats
from nba_api.stats.static import players, teams
import json
import requests
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class NbaScraper:
    """ Class to scrape data from the NBA official website.
    """
    @staticmethod
    def get_json_from_name(name: str, is_player=True) -> int:
        """ Get the json of a player or team from his name
        """
        from nba_api.stats.static import players, teams
        if is_player:
            nba_players = players.get_players()
            return [player for player in nba_players 
                    if player['full_name'] == name][0]
        else:
            nba_teams = teams.get_teams()
            return [team for team in nba_teams 
                    if team['full_name'] == name][0]
    
    @staticmethod
    def get_player_career(player_id: int) -> list:
        """ Get the career of a player from his id
        """
        from nba_api.stats.endpoints import playercareerstats
        career = playercareerstats.PlayerCareerStats(player_id=player_id)
        return career.get_data_frames()[0]
    
    @staticmethod
    def get_shot_data(id: int, team_ids: list, seasons: list) -> list:
        """ Get the shot data of a player from his id and seasons
        """
        from nba_api.stats.endpoints import shotchartdetail
        df = pd.DataFrame()
        for season in seasons:
            for team in team_ids:
                shot_data = shotchartdetail.ShotChartDetail(
                    team_id=team,
                    player_id=id,
                    context_measure_simple='FGA',
                    season_nullable=season
                )
                df = pd.concat([df, shot_data.get_data_frames()[0]])
        
        return df
    
    @staticmethod
    def get_all_ids(only_active=True) -> list:
        """ Get all the ids of the players
        """
        from nba_api.stats.static import players
        nba_players = players.get_players()
        if only_active:
            return [player['id'] for player in nba_players 
                    if player['is_active']]
        return [player['id'] for player in nba_players]
    
    @staticmethod
    def get_player_headshot(id: int) -> str:
            """ Get the headshot of a player from his id
            """
            from nba_api.stats.static import players
            import requests
            import shutil
            
            url = f'https://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/{id}.png'
            output_path = f'../data/nba/transient/headshots/{id}.png'
            r = requests.get(url, stream=True)
            if r.status_code == 200:
                with open(output_path, 'wb') as f:
                    r.raw.decode_content = True
                    shutil.copyfileobj(r.raw, f)
    
    @staticmethod                                    
    def get_all_nba_headshots(only_active=False) -> None:
        """ Get the headshots of all the players
        """
        ids = NbaScraper.get_all_ids(only_active=only_active)
        for id in ids:
            NbaScraper.get_player_headshot(id)

### Ktaegorije: sezona, posamezna tekma, posamezna ekipa, čas tekme, pozicija meta

In [9]:
# get player id
name = "Luka Doncic"
id = NbaScraper.get_json_from_name(name)['id']
print(id)
career = NbaScraper.get_player_career(id)
teams = ['DAL']
team_ids = list(set(career[career.TEAM_ABBREVIATION.isin(teams)].TEAM_ID.values))

seasons = ['2018-19','2019-20','2020-21','2021-22','2022-23','2023-24','2024-25']
shot_data = NbaScraper.get_shot_data(id, team_ids, seasons)
print(len(shot_data))
shot_data.tail()

1629029
8510


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
209,Shot Chart Detail,0022400190,528,1629029,Luka Dončić,1610612742,Dallas Mavericks,4,8,51,...,Center(C),Less Than 8 ft.,4,47,15,1,1,20241108,DAL,PHX
210,Shot Chart Detail,0022400190,534,1629029,Luka Dončić,1610612742,Dallas Mavericks,4,8,13,...,Center(C),8-16 ft.,11,13,118,1,0,20241108,DAL,PHX
211,Shot Chart Detail,0022400190,563,1629029,Luka Dončić,1610612742,Dallas Mavericks,4,5,2,...,Center(C),8-16 ft.,13,-29,136,1,0,20241108,DAL,PHX
212,Shot Chart Detail,0022400190,603,1629029,Luka Dončić,1610612742,Dallas Mavericks,4,1,13,...,Left Side Center(LC),24+ ft.,25,-167,197,1,0,20241108,DAL,PHX
213,Shot Chart Detail,0022400190,639,1629029,Luka Dončić,1610612742,Dallas Mavericks,4,0,0,...,Left Side Center(LC),24+ ft.,37,-158,342,1,0,20241108,DAL,PHX


In [ ]:
# Assuming shot_data is a pandas DataFrame
shot_data.to_csv('../data/shot_data.csv', index=False)